In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('klee-results/pending_reproduction.csv',
                   header=0, skipfooter=1, engine='python')
data

In [ ]:
benchmarks_lst = [
    'bc', 'datamash', 'm4', 'magick-jpg', 'magick-png',
    'make', 'oggenc', 'sqlite3', 'tcpdump'
]
search_lst = ['dfs', 'nurs:depth', 'random-path']

In [ ]:
benchmarks = list(map(lambda x: benchmarks_lst[x // 3], list(data.index)))
searches = list(map(lambda x: search_lst[x % 3], list(data.index)))
data['benchmark'] = benchmarks
data['search'] = searches
processed_data = data.set_index(['benchmark', 'search'])

In [ ]:
solver_data = processed_data['TSolver(%)'].unstack(level=1)
solver_data

In [ ]:
solver_data.plot.bar(rot=45, figsize=(15, 10))

In [ ]:
solver_data

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 15}
matplotlib.rc('font', **font)

labels = solver_data.index
random_path = solver_data['random-path']
nurs_depth = solver_data['nurs:depth']
dfs = solver_data['dfs']

x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

fig, axes = plt.subplots(
    2, 1, sharex=True, figsize=(15, 10),
    gridspec_kw={'height_ratios': [3, 1]})

for ax in axes:
    ax.bar(x - width, random_path, width, label='RP')
    ax.bar(x, nurs_depth, width, label='NURS:DEPTH')
    ax.bar(x + width, dfs, width, label='dfs')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45)

axes[0].set_ylabel('TSolver(%)', loc='bottom')
axes[0].set_ylim(65., 100.)  # outliers only
axes[1].set_ylim(0., 5.)  # most of the data

# hide the spines between ax and ax2
axes[0].spines['bottom'].set_visible(False)
axes[1].spines['top'].set_visible(False)
axes[0].xaxis.tick_top()
axes[1].tick_params(labeltop=False)  # don't put tick labels at the top
axes[1].xaxis.tick_bottom()

d = 0.005  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=axes[0].transAxes, color='k', clip_on=False)
axes[0].plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
axes[0].plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=axes[1].transAxes)  # switch to the bottom axes
axes[1].plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
axes[1].plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

plt.savefig('tsolver.png', dpi=300)
plt.show()